<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/CLASSIFICATION1_LLMembed_shkim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install torch transformers tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset, ClassLabel, DatasetDict

agnews_dataset = load_dataset("fancyzhx/ag_news")
print(agnews_dataset['train'].features)

new_labels = agnews_dataset["train"].features["label"].names + ["StackOverFlow"]
new_class_label = ClassLabel(names=new_labels)

# 새로운 클래스 레이블을 기존 데이터셋에 적용하기 위해 데이터셋을 업데이트합니다.
updated_features = agnews_dataset["train"].features.copy()
updated_features["label"] = new_class_label

agnews_dataset = agnews_dataset.cast(updated_features)

# 결과 출력
print(agnews_dataset['train'].features)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)}


Casting the dataset:   0%|          | 0/120000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech', 'StackOverFlow'], id=None)}


In [7]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, concatenate_datasets
from datasets import ClassLabel, Value

sof_dataset_train = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/convert_train.csv',
    split='train'
)
# print(sof_dataset_train)

class_label = ClassLabel(num_classes=5, names=['World', 'Sports', 'Business', 'Sci/Tech', 'StackOverFlow'])

# 데이터셋에서 30000건 추출
sof_train_samples = sof_dataset_train.shuffle(seed=42).select(range(30000))
# 'Concat_Text' 필드를 'text'로, 그리고 'label' 값을 'StackOverFlow'로 설정
sof_train_samples = sof_train_samples.map(lambda example: {'text': example['Concat_Text'], 'label': class_label.str2int('StackOverFlow')}, remove_columns=['Id', 'Title', 'Body', 'Tags', 'CreationDate', 'Y', 'Cleaned_tag ', 'Cleaned_tag', 'Array_Tag', 'Filtered_Array_Tag', 'Filtered_Tag', 'Concat_Text'])

# sof_train_samples의 features를 업데이트
sof_train_samples = sof_train_samples.cast_column('label', class_label)

# sof_train_samples와 agnews_train을 결합
combined_train_dataset = concatenate_datasets([agnews_dataset["train"], sof_train_samples])

# 결과를 확인
print(combined_train_dataset)

# 레이블의 데이터 건수 카운트
label_counts = combined_train_dataset.features['label'].num_classes
label_counter = {label: 0 for label in range(label_counts)}

for example in combined_train_dataset:
    label_counter[example['label']] += 1

print(label_counter)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/30000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 150000
})
{0: 30000, 1: 30000, 2: 30000, 3: 30000, 4: 30000}


In [8]:
sof_dataset_test = load_dataset(
    'csv',
    data_files='/content/drive/MyDrive/LLMEmbed/convert_valid.csv',
    split='train'
)
sof_test_samples = sof_dataset_test.shuffle(seed=42).select(range(1900))
# 'Concat_Text' 필드를 'text'로, 그리고 'label' 값을 'StackOverFlow'로 설
sof_test_samples = sof_test_samples.map(lambda example: {'text': example['Concat_Text'], 'label': class_label.str2int('StackOverFlow')}, remove_columns=['Id', 'Title', 'Body', 'Tags', 'CreationDate', 'Y', 'Cleaned_tag ', 'Cleaned_tag', 'Array_Tag', 'Filtered_Array_Tag', 'Filtered_Tag', 'Concat_Text'])

# sof_train_samples의 features를 업데이트
sof_test_samples = sof_test_samples.cast_column('label', class_label)

combined_test_dataset = concatenate_datasets([agnews_dataset["test"], sof_test_samples])

# 결과 확인
print(combined_test_dataset)

# 레이블의 데이터 건수 카운트
label_counts = combined_test_dataset.features['label'].num_classes
label_counter = {label: 0 for label in range(label_counts)}

for example in combined_test_dataset:
    label_counter[example['label']] += 1

print(label_counter)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1900 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1900 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 9500
})
{0: 1900, 1: 1900, 2: 1900, 3: 1900, 4: 1900}


In [9]:
from transformers import AutoTokenizer

new_agnews_dataset= DatasetDict({
    "train": combined_train_dataset,
    "test": combined_test_dataset
})

# DatasetDict 출력
print(new_agnews_dataset)

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = new_agnews_dataset.map(tokenize_function, batched=True)



small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1500))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1500))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 9500
    })
})


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

# **1. Llama2 Embedding**

In [10]:
# -*- coding: utf-8 -*-
import os
import torch
import json
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from tqdm import trange
from datasets import load_dataset
import argparse

def rep_extract(task, mode, device, sents, labels, max_len, step):
    model_id = "daryl149/llama-2-7b-chat-hf"

    # model_id = "kimnt93/chat-llama2-1b-1.0"

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = "[PAD]"
    tokenizer.padding_side = "right"

    config_kwargs = {
        "trust_remote_code": True,
        "cache_dir": None,
        "revision": 'main',
        "use_auth_token": None,
        "output_hidden_states": True
    }
    model_config = AutoConfig.from_pretrained(model_id, **config_kwargs)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        config=model_config,
        device_map=device,
        torch_dtype=torch.float16)
    model.eval()

    sents_reps = []
    # for idx in trange(0, 20, step):
    for idx in trange(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)
        sents_batch = sents[idx: idx_end]

        sents_batch_encoding = tokenizer(sents_batch, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
        sents_batch_encoding = sents_batch_encoding.to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)

            reps_batch_5L = []
            for layer in range(-1, -6, -1):
                reps_batch_5L.append(torch.mean(batch_outputs.hidden_states[layer], axis=1))
            reps_batch_5L = torch.stack(reps_batch_5L, axis=1)

        sents_reps.append(reps_batch_5L.cpu())
    sents_reps = torch.cat(sents_reps)

    for idx in range(len(labels)):
        labels[idx] = torch.tensor(labels[idx])
    labels = torch.stack(labels)

    print(sents_reps.shape)
    print(labels.shape)
    path = f'/content/drive/MyDrive/LLMEmbed/llama2/{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
    torch.save(labels, path + f'{mode}_labels.pt')

if __name__ == '__main__':
    cuda_no = 0 # GPU 번호
    task = 'agnews' # Task 이름
    device = f'cuda:{cuda_no}'

    # dataset = load_dataset("dxgp/R8")
    sents = new_agnews_dataset['train']['text']
    labels = new_agnews_dataset['train']['label']
    rep_extract(task, 'train', device, sents, labels, 256, 40)

    sents = new_agnews_dataset['test']['text']
    labels = new_agnews_dataset['test']['label']
    rep_extract(task, 'test', device, sents, labels, 256, 40)


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


...

100%|██████████| 3750/3750 [48:56<00:00,  1.28it/s]


torch.Size([150000, 5, 4096])
torch.Size([150000])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 238/238 [03:06<00:00,  1.28it/s]


torch.Size([9500, 5, 4096])
torch.Size([9500])


# **2. BERT Embedding**

In [ ]:
# -*- coding: utf-8 -*-
import os
import torch
import json
from transformers import BertTokenizer, BertModel
from tqdm import trange
from datasets import load_dataset
import argparse

def rep_extract(task, mode, device, sents, labels):
    model_path = 'google-bert/bert-large-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertModel.from_pretrained(model_path).to(device)
    model.eval()

    max_len = 512
    sents_reps = []
    step = 512
    for idx in trange(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)
        sents_batch = sents[idx: idx_end]

        sents_batch_encoding = tokenizer(sents_batch, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
        sents_batch_encoding = sents_batch_encoding.to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)
            reps_batch = batch_outputs.pooler_output
        sents_reps.append(reps_batch.cpu())
    sents_reps = torch.cat(sents_reps)

    for idx in range(len(labels)):
        labels[idx] = torch.tensor(labels[idx])
    labels = torch.stack(labels)

    print(sents_reps.shape)
    print(labels.shape)
    path = f'/content/drive/MyDrive/LLMEmbed/bert/{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
    torch.save(labels, path + f'{mode}_labels.pt')

if __name__ == '__main__':

    cuda_no = 0 # GPU 번호
    task = 'agnews' # Task 이름
    device = f'cuda:{cuda_no}'

    # dataset = load_dataset("dxgp/R8")
    print(new_agnews_dataset)
    print(new_agnews_dataset['train'][0])
    sents = new_agnews_dataset['train']['text']

    labels = new_agnews_dataset['train']['label']
    rep_extract(task, 'train', device, sents, labels)

    sents = new_agnews_dataset['test']['text']
    labels = new_agnews_dataset['test']['label']
    rep_extract(task, 'test', device, sents, labels)



DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 9500
    })
})
{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

100%|██████████| 293/293 [50:51<00:00, 10.41s/it]


torch.Size([150000, 1024])
torch.Size([150000])


100%|██████████| 19/19 [03:13<00:00, 10.17s/it]

torch.Size([9500, 1024])
torch.Size([9500])


# **3. Roberta Embedding**

In [ ]:
# -*- coding: utf-8 -*-
import os
import torch
import json
from transformers import RobertaTokenizer, RobertaModel
from tqdm import trange
from datasets import load_dataset
import argparse

def rep_extract(task, mode, device, sents, labels):
    model_path = 'FacebookAI/roberta-large'
    tokenizer = RobertaTokenizer.from_pretrained(model_path)
    model = RobertaModel.from_pretrained(model_path).to(device)
    model.eval()

    max_len = 512
    sents_reps = []
    step = 512
    for idx in trange(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)
        sents_batch = sents[idx: idx_end]

        sents_batch_encoding = tokenizer(sents_batch, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
        sents_batch_encoding = sents_batch_encoding.to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)
            reps_batch = batch_outputs.last_hidden_state[:, 0, :]
        sents_reps.append(reps_batch.cpu())
    sents_reps = torch.cat(sents_reps)

    for idx in range(len(labels)):
        labels[idx] = torch.tensor(labels[idx])
    labels = torch.stack(labels)

    print(sents_reps.shape)
    print(labels.shape)
    path = f'/content/drive/MyDrive/LLMEmbed/roberta/{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
    torch.save(labels, path + f'{mode}_labels.pt')

if __name__ == '__main__':
    cuda_no = 0 # GPU 번호
    task = 'agnews' # Task 이름
    device = f'cuda:{cuda_no}'

    # dataset = load_dataset("dxgp/R8")
    sents = new_agnews_dataset['train']['text']
    labels = new_agnews_dataset['train']['label']
    rep_extract(task, 'train', device, sents, labels)

    sents = new_agnews_dataset['test']['text']
    labels = new_agnews_dataset['test']['label']
    rep_extract(task, 'test', device, sents, labels)


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 293/293 [54:49<00:00, 11.23s/it]


torch.Size([150000, 1024])
torch.Size([150000])


Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 19/19 [03:29<00:00, 11.03s/it]

torch.Size([9500, 1024])
torch.Size([9500])


# **4. Dataset 정의**

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):

    def __init__(self, mode, l_path, b_path, r_path):
        self.l_sents_reps = torch.load(l_path + f'{mode}_sents.pt')
        self.b_sents_reps = torch.load(b_path + f'{mode}_sents.pt')
        self.r_sents_reps = torch.load(r_path + f'{mode}_sents.pt')

        self.labels = torch.load(l_path + f'{mode}_labels.pt')

        self.sample_num = self.labels.shape[0]

    def __getitem__(self, index):
        return self.l_sents_reps[index], self.b_sents_reps[index], self.r_sents_reps[index], self.labels[index]

    def __len__(self):
        return self.sample_num

# **5. Model operation (multi)**

In [2]:
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm
import numpy as np

# epoch - Train
def Train_multi(dataloader, device, model, loss_fn, optimizer):
    loss_list, acc_list, micro_f1_list, macro_f1_list = [], [], [], []

    for batch_i, batch_loader in enumerate(tqdm(dataloader)):
        batch_l, batch_b, batch_r, batch_y = batch_loader
        batch_l, batch_b, batch_r, batch_y = batch_l.to(device), batch_b.to(device), batch_r.to(device), batch_y.to(device)

        model.train()
        pred = model(batch_l.float(), batch_b.float(), batch_r.float())
        loss = loss_fn(pred, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred_y = torch.max(pred, 1).indices
        # print(pred_y)
        acc = accuracy_score(batch_y.cpu(), pred_y.cpu())
        micro_f1 = f1_score(batch_y.cpu(), pred_y.cpu(), average='micro')
        macro_f1 = f1_score(batch_y.cpu(), pred_y.cpu(), average='macro')
        loss = loss.cpu()

        loss_list.append(loss.item())
        acc_list.append(acc)
        micro_f1_list.append(micro_f1)
        macro_f1_list.append(macro_f1)
    print(f'loss: {np.mean(loss_list):.4f}')
    print(f'acc: {np.mean(acc_list):.4f}')
    print(f'micro_f1: {np.mean(micro_f1_list):.4f}')
    print(f'macro_f1: {np.mean(macro_f1_list):.4f}')

def Test_multi(dataloader, device, model, loss_fn):
    avg_loss = 0
    total_pred, total_y = [], []

    for batch_i, batch_loader in enumerate(tqdm(dataloader)):
        batch_l, batch_b, batch_r, batch_y = batch_loader
        batch_l, batch_b, batch_r, batch_y = batch_l.to(device), batch_b.to(device), batch_r.to(device), batch_y.to(device)

        model.eval()
        with torch.no_grad():
            pred = model(batch_l.float(), batch_b.float(), batch_r.float())
            loss = loss_fn(pred, batch_y)
            loss = loss.to('cpu')
            avg_loss += loss.item()

        pred_y = torch.max(pred, 1).indices
        total_pred.append(pred_y.cpu())
        total_y.append(batch_y.cpu())

    avg_loss = avg_loss / (batch_i+1)

    total_y = torch.cat(total_y)
    total_pred = torch.cat(total_pred)
    acc = accuracy_score(total_y, total_pred)
    micro_f1 = f1_score(total_y.cpu(), total_pred.cpu(), average='micro')
    macro_f1 = f1_score(total_y.cpu(), total_pred.cpu(), average='macro')

    print(f'avg loss: {avg_loss:.4f}')
    print(f'acc: {acc:.4f}')
    print(f'micro_f1: {micro_f1:.4f}')
    print(f'macro_f1: {macro_f1:.4f}')

# **6. Downstream Model Class**

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DownstreamModel(nn.Module):
    def __init__(self, class_num, SIGMA):
        super(DownstreamModel, self).__init__()
        self.SIGMA = SIGMA
        self.compress_layers = nn.ModuleList()
        for _ in range(5):
            layers = []
            layers.append(nn.Linear(4096, 1024))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.5))
            self.compress_layers.append(nn.Sequential(*layers))

        self.fc1 = nn.Linear(4145, 1024)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, class_num)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_l, input_b, input_r):
        batch_size = input_l.shape[0]
        split_tensors = torch.split(input_l, 1, dim=1)
        input = []
        for i, split_tensor in enumerate(split_tensors):
            split_tensor = split_tensor.reshape(batch_size,-1)
            input.append(self.compress_layers[i](split_tensor))
        input.append(input_b)
        input.append(input_r)
        input = torch.stack(input, dim=1)
        # X * X^T
        input_T = input.transpose(1, 2)
        input_P = torch.matmul(input, input_T)
        input_P = input_P.reshape(batch_size, -1)
        # PN func
        input_P = 2*F.sigmoid(self.SIGMA * input_P) - 1

        a = torch.mean(input_l, dim=1)
        input = torch.cat([input_P, a], dim=1)
        # print(input.shape)

        output = self.fc1(input)
        output = self.relu1(output)
        output = self.dropout1(output)
        output = self.fc2(output)
        output = self.relu2(output)
        output = self.dropout2(output)
        output = self.fc3(output)
        output = self.softmax(output)

        return output

# if __name__ == '__main__':
#     model = DownstreamModel(2)
#     print(model)

# **7. Fine-tuning**

In [22]:
# from DownstreamModel import DownstreamModel
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
# from model_op import Train, Test
# from model_op_multi import Train_multi, Test_multi
import argparse
import os
import torch
# from MyDataset import MyDataset
import json

if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cuda_no = 0  # 예를 들어, '0'을 사용
    task = 'agnews'  # 사용 가능한 옵션: 'sst2', 'mr', 'agnews', 'r8', 'r52' 중 하나
    epoches = 100  # 원하는 에폭 수
    SIGMA = 0.1  # SIGMA 값을 설정
    batch_size = 1024  # 배치 크기 설정
    lr = 1e-4  # 학습률 설정

    class_num = {'sst2':2, 'mr':2, 'agnews':5, 'r8':8, 'r52':52}
    class_num = class_num[task]

    l_dataset_path = f'/content/drive/MyDrive/LLMEmbed/llama2/{task}/dataset_tensor/'
    b_dataset_path = f'/content/drive/MyDrive/LLMEmbed/bert/{task}/dataset_tensor/'
    r_dataset_path = f'/content/drive/MyDrive/LLMEmbed/roberta/{task}/dataset_tensor/'
    mode = 'train'
    train_data = MyDataset(mode, l_dataset_path, b_dataset_path, r_dataset_path)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    mode = 'test'
    test_data = MyDataset(mode, l_dataset_path, b_dataset_path, r_dataset_path)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    model = DownstreamModel(class_num, SIGMA).to(device)

    loss_fn = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr)

    if class_num == 2:
        print('training ...')
        for epoch in range(epoches):
            model = model.to(device)
            print(f'--------------------------- epoch {epoch} ---------------------------')
            Train(train_loader, device, model, loss_fn, optimizer)
        print('save model_wights')
        # 모델 학습 후, 모델의 가중치 저장하기
        model_save_path = "/content/drive/MyDrive/LLMEmbed/model_weights.pth"
        torch.save(model.state_dict(), model_save_path)
        print('evaluate ...')
        Test(test_loader, device, model, loss_fn)

    # multi-class
    elif class_num > 2:
        print('training ...')
        for epoch in range(epoches):
            model = model.to(device)
            print(f'--------------------------- epoch {epoch} ---------------------------')
            Train_multi(train_loader, device, model, loss_fn, optimizer)
        print()
        # 모델 학습 후, 모델의 가중치 저장하기
        model_save_path = "/content/drive/MyDrive/LLMEmbed/model_weights_agnews.pth"
        torch.save(model.state_dict(), model_save_path)
        print('evaluate ...')
        Test_multi(test_loader, device, model, loss_fn)

<ipython-input-1-9044650157bc>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.l_sents_reps = torch.load(l_path + f'{mode}_sents.pt')
<ipython-input-1-9044650157bc>:8:

training ...
--------------------------- epoch 0 ---------------------------


100%|██████████| 147/147 [00:05<00:00, 27.81it/s]


loss: 1.0586
acc: 0.8725
micro_f1: 0.8725
macro_f1: 0.8697
--------------------------- epoch 1 ---------------------------


100%|██████████| 147/147 [00:05<00:00, 27.90it/s]


...

loss: 0.9255
acc: 0.9795
micro_f1: 0.9795
macro_f1: 0.9794
--------------------------- epoch 99 ---------------------------


100%|██████████| 147/147 [00:05<00:00, 27.35it/s]


loss: 0.9249
acc: 0.9801
micro_f1: 0.9801
macro_f1: 0.9800

evaluate ...


100%|██████████| 10/10 [00:01<00:00,  7.49it/s]

avg loss: 0.9410
acc: 0.9608
micro_f1: 0.9608
macro_f1: 0.9608


# **8. 추론 메서드 정의**

In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BertTokenizer, BertModel, RobertaTokenizer, RobertaModel
from transformers import AutoConfig

# Load the tokenizers and models for Llama2, BERT, and Roberta
llama2_tokenizer = AutoTokenizer.from_pretrained("daryl149/llama-2-7b-chat-hf", trust_remote_code=True)
llama2_tokenizer.pad_token = llama2_tokenizer.eos_token  # 패딩 토큰 설정
llama2_config = AutoConfig.from_pretrained("daryl149/llama-2-7b-chat-hf", output_hidden_states=True)
llama2_model = AutoModelForCausalLM.from_pretrained("daryl149/llama-2-7b-chat-hf", config=llama2_config)

bert_tokenizer = BertTokenizer.from_pretrained('google-bert/bert-large-uncased')
bert_model = BertModel.from_pretrained('google-bert/bert-large-uncased')

roberta_tokenizer = RobertaTokenizer.from_pretrained('FacebookAI/roberta-large')
roberta_model = RobertaModel.from_pretrained('FacebookAI/roberta-large')

# Make sure all models are in evaluation mode and moved to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
llama2_model.eval().to(device)
bert_model.eval().to(device)
roberta_model.eval().to(device)

# Initialize the downstream model
class_num = 5  # For example, if you have 8 classes
SIGMA = 0.1  # SIGMA value for your downstream model
downstream_model = DownstreamModel(class_num, SIGMA).to(device)

model_load_path = "/content/drive/MyDrive/LLMEmbed/model_weights_agnews.pth"

# 가중치 로드
downstream_model.load_state_dict(torch.load(model_save_path, map_location=device))
downstream_model.eval()

def get_llama2_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        # Average the last 5 layers
        embedding = torch.stack([torch.mean(outputs.hidden_states[i], dim=1) for i in range(-1, -6, -1)], dim=1)
    return embedding

def get_bert_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        # Use pooler_output for BERT embeddings
        embedding = outputs.pooler_output
    return embedding

def get_roberta_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        # Use the first token ([CLS] token) representation
        embedding = outputs.last_hidden_state[:, 0, :]
    return embedding

def infer(text, downstream_model, device):
    # 각 모델로부터 임베딩을 추출
    llama2_emb = get_llama2_embedding(text, llama2_tokenizer, llama2_model, device)
    bert_emb = get_bert_embedding(text, bert_tokenizer, bert_model, device)
    roberta_emb = get_roberta_embedding(text, roberta_tokenizer, roberta_model, device)

    # Forward pass through the downstream model
    with torch.no_grad():
        prediction = downstream_model(llama2_emb, bert_emb, roberta_emb)

    return prediction


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-23-dde813e42baf>:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com

# **9. 추론 테스트**

In [24]:
label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech",
    4: "StackOverFlow"
}

# 주어진 텍스트를 토대로 예측을 5번 실행
texts = [
    "US trade deficit swells in June The US trade deficit has exploded 19 to a record \$55.8bn as oil costs drove imports higher, according to a latest figures.",
    "Indians Beat Twins 7-1, Nearing AL Lead (AP) AP - The Cleveland Indians pulled within one game of the AL Central lead, scoring four runs in the first inning and beating the Minnesota Twins 7-1 Saturday night behind home runs by Travis Hafner and Victor Martinez.",
    "Pilgrims Crowd Field for Mass With Pope LOURDES, France - A frail Pope John Paul II celebrated an open-air Mass on Sunday as several hundred thousand pilgrims, many in wheelchairs, crowded onto a field near a French shrine to the Virgin Mary that is associated with miraculous cures of the sick. The Mass was a highlight of the Pope's two-day visit to Lourdes, a town in the Pyrenees where Roman Catholic tradition says St...",
    "T. Rex Had Teen Growth Spurt, Scientists Say (Reuters) Reuters - Tyrannosaurus Rex grew incredibly fast\during a teenaged growth spurt that saw the dinosaur expand its\bulk by six times, but the fearsome beasts \"lived fast and died\young,\" researchers said on Wednesday.",
    "working of compareTo() method of Comparable interface            I have one Employee class and the requirement is to sort the objects using comparable interface. The output with this code is :The difference of this id and other id is..** 6  other id**1The difference of this id and other id is..** 3  other id**6The difference of this id and other id is..** 3  other id**6The difference of this id and other id is..** 3  other id**1The difference of this id and other id is..** 11  other id**3The difference of this id and other id is..** 11  other id**6",
]

# 각 텍스트에 대해 예측 실행 및 레이블 출력
for i, text in enumerate(texts):
    prediction = infer(text, downstream_model, device)
    predicted_label = torch.argmax(prediction, dim=1).item()
    predicted_score = torch.max(prediction).item()
    label_name = label_map[predicted_label]
    print(f"Text {i+1} Predicted label: {label_name}, Score: {predicted_score:.4f}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Text 1 Predicted label: Business, Score: 1.0000
Text 2 Predicted label: Sports, Score: 0.9954
Text 3 Predicted label: World, Score: 1.0000
Text 4 Predicted label: Sci/Tech, Score: 1.0000
Text 5 Predicted label: StackOverFlow, Score: 1.0000
